# nanoGPT Development

In [2]:
import torch

In [ ]:
# Read in data, inspect and convert to mapping

with open("input.txt", "r") as infile:
    text = infile.read()
print(f"Dataset length: {len(text)}")

chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(f"Vocab Size: {vocab_size}")

In [ ]:
# Create mapping from characters to integers (character level tokenizer)

STOI = {character: integer for integer, character in enumerate(chars)}
ITOS = {integer: character for integer, character in enumerate(chars)}


def encode(input_: str) -> "list[int]":
    return [STOI[char] for char in input_]


def decode(input_: "list[int]") -> str:
    return "".join([ITOS[integer] for integer in input_])


message = "Hello, there!"
print(f"Encode: {message}: {encode(message)}")
print(f"Decode: Encode({message}): {decode(encode(message))}")

In [ ]:
# Compare the above with OpenAI tiktoken encoding (much larger vocabulary)
import tiktoken

encoding = tiktoken.get_encoding("gpt2")

print(f"tiktoken Vocab Size: {encoding.n_vocab}")
print(f"tiktoken Encode: {encoding.encode(message)}")
print(f"tiktoken Decode: {encoding.decode(encoding.encode(message))}")

In [ ]:
# Convert input data to torch tensor

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

In [ ]:
# Convert the data into a train/test split
n = int(0.9 * len(data))
train_data = data[:n]
test_data = data[n:]

print(f"Num train examples: {train_data.shape}")
print(f"Num test examples: {test_data.shape}")

In [ ]:
block_size = 8

train_data[: block_size + 1]

In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8


def get_batch(split: str):
    """Generate a mini-batch of training data."""
    if split not in ["train", "test"]:
        raise ValueError("`split` must be one of `train` or `test`.")

    data = train_data if split == "train" else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    return x, y


xb, yb = get_batch("train")

for b in range(batch_size):  # batch dimension
    for t in range(block_size):  # time dimension
        context = xb[b, : t + 1]
        target = yb[b, t]
        print(f"When the input is {context} the target is {target}")
        print(f"{decode(context.tolist())} -> {decode([target.item()])}")

In [36]:
# Begin with the simplest possible model - bigram language model
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        """Forward pass. If targets is none, just return the logits."""
        # idx and targets are both (B, T) batches of integers
        logits = self.token_embedding_table(idx)  # (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)  # cross entropy expects a (B, C, d...)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        """Sample from the model."""
        pass

In [34]:
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)